# Tensorboard

---

# 1 Tensorboard 활용
- [Tensorboard](https://www.tensorflow.org/tensorboard/get_started)
1. tensorboard 종료: 작업관리자 -> tessorboard.exe 작업 끝내기
2. 로그 폴더 삭제: 관리자 권한 CDM -> rmdir /s /q logs

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

In [ ]:
#!pip install jupyter-tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./logs

### 1.1 보스턴 집값 예측 - Regression

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

import tensorflow as tf

In [ ]:
# seed 값 설정
np.random.seed(0)
tf.random.set_seed(3)

#### 데이터 로드

In [ ]:
df = pd.read_csv('./data/housing.csv', delim_whitespace=True, header=None)
dataset = df.values
X = dataset[:,0:13]
y = dataset[:,13]

#### 데이터 분할

In [ ]:
# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#### 모델 생성 및 설정

In [ ]:
# 모델 생성 및 설정
model_b = Sequential()
model_b.add(Dense(30, input_dim=13, activation='relu'))
model_b.add(Dense(6, activation='relu'))
model_b.add(Dense(1))

#### 모델 컴파일

In [ ]:
# 모델 컴파일
model_b.compile(loss='mean_squared_error', optimizer='adam')

#### 텐서보드 콜백 함수 설정

In [ ]:
# 시간정보를 활용하여 폴더 생성
import datetime
# 학습데이터의 log를 저장할 폴더 생성 (지정)
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

# 텐서보드 콜백 정의 하기
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### 모델 학습 실행

In [ ]:
%%time
model_b.fit(X_train, y_train, epochs=200, batch_size=10, callbacks=[tensorboard_callback])

### 1.2 와인 종류 예측 - Classification

[Data - UCI repository](https://archive.ics.uci.edu/ml/datasets/wine)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping

import tensorflow as tf

In [ ]:
# seed 값 설정
np.random.seed(3)
tf.random.set_seed(3)

#### 데이터 로드 - 20%

In [ ]:
df_pre = pd.read_csv('./data/wine.csv', header=None)
df = df_pre.sample(frac=0.2) # 20%

dataset = df.values
X = dataset[:,0:12]
y = dataset[:,12]

#### 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#### 모델 생성 및 설정

In [ ]:
model_w = Sequential()
model_w.add(Dense(30,  input_dim=12, activation='relu'))
model_w.add(Dense(12, activation='relu'))
model_w.add(Dense(8, activation='relu'))
model_w.add(Dense(1, activation='sigmoid'))

#### 모델 컴파일

In [ ]:
model_w.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 모델 업데이트 및 저장 설정

In [ ]:
modelpath='./model/model_wine_{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

#### 학습 자동 중단 설정

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

#### 텐서보드 콜백 함수 설정

In [ ]:
# 시간정보를 활용하여 폴더 생성
import datetime
# 학습데이터의 log를 저장할 폴더 생성 (지정)
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

# 텐서보드 콜백 정의 하기
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### 모델 학습 실행 및 저장

In [ ]:
%%time
history = model_w.fit(X_train, y_train, validation_split=0.2, epochs=3500, batch_size=500, verbose=0,
                      callbacks=[early_stopping_callback,checkpointer,tensorboard_callback])

---

In [ ]:
# End of file